In [1]:
# github.com/songys/Chatbot_data 를 활용한 챗봇 만들어보기

import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [3]:
# 데이터 읽어오기
train_file = "./ChatbotData.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

MAX_SEQ_LEN = 15
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')


In [8]:
# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋 : 검증셋 : 테스트셋 = 7 : 2 : 1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

# print(train_size + val_size)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [13]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1

In [14]:
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN, ))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters=128,
               kernel_size=3,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(filters=128,
               kernel_size=4,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(filters=128,
               kernel_size=5,
               padding='valid',
               activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [15]:
# 3, 4, 5-gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

In [16]:
# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
414/414 [==============================] - 6s 12ms/step - loss: 0.9116 - accuracy: 0.5456 - val_loss: 0.7153 - val_accuracy: 0.6747
Epoch 2/5
414/414 [==============================] - 5s 12ms/step - loss: 0.5737 - accuracy: 0.7672 - val_loss: 0.3439 - val_accuracy: 0.8968
Epoch 3/5
414/414 [==============================] - 5s 11ms/step - loss: 0.3339 - accuracy: 0.8838 - val_loss: 0.1752 - val_accuracy: 0.9492
Epoch 4/5
414/414 [==============================] - 5s 11ms/step - loss: 0.2131 - accuracy: 0.9328 - val_loss: 0.0872 - val_accuracy: 0.9729
Epoch 5/5
414/414 [==============================] - 5s 12ms/step - loss: 0.1331 - accuracy: 0.9598 - val_loss: 0.0648 - val_accuracy: 0.9797


In [18]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy : %.2f' %(accuracy * 100))
print('loss : %.2f' %(loss))

# 모델 저장
model.save('cnn_model.h5')

60/60 [==============================] - 0s 1ms/step - loss: 0.0646 - accuracy: 0.9780
Accuracy : 97.80
loss : 0.06
